<a href="https://colab.research.google.com/github/KARENCMP82/Python/blob/main/4_PCA_CF_ANIME_RRB_26MAR25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nuclio Digital School - Máster en Data Science**

# Unsupervised Learning: Reducción de la dimensionalidad y Collaborative filtering

# *Profesora: Raquel Revilla*

<a id = "objetivos"></a>
# Objetivos del notebook
[Volver al índice](#toc)

Una de las áreas del machine learning con la cual interactuamos casi a diario son los modelos del Collaborative Filtering. Cada vez que nos conectamos a Instagram, Facebook, Amazon recibimos un feed personalizado de productos o servicios. En el notebook de esta sección, vamos a crear un feed personalizado para los fans de animes: **un feed estará basando en modelos de ML (KMeans)** y **otro feed será basado en la similitud de coseno.**

El dataset que vamos a utilizar es un dataset de Animes japoneses y se puede descargar en el siguiente [enlace](https://www.kaggle.com/CooperUnion/anime-recommendations-database)

![Data Model](https://drive.google.com/uc?export=view&id=11b0WFJxHM3R9Jm5ATir5f3OA_0h78Mu2)

Nuestros principales objetivos serán:
1. **Hacer una exploración inicial de los dos datasets** y entender la distribución de los datos.



2. **Procesar el dataset (eliminar usuarios sin puntuaciones).**



3. **Reducir la dimensionalidad de nuestro DataFrame utilizando el PCA.**


4. **Segmentar nuestros clientes utilizando el dataset reducido.**


5. **Utilizar la similitud del coseno para hacer recomendaciones a nuestro clientes (user and product based).**

<a id = "toc"></a>
# Índice

[Importación de las principales librerías](#import_modules)

[Importación de los datos](#import_data)

[Exploratory Data Analysis (EDA)](#eda)

---> [EDA anime df](#df1)

---> [EDA ratings df](#df2)

[Join final animes con ratings de usuarios](#join)

[Reducción de la dimensionalidad con PCA](#pca)

[Elbow curve, KMeans y recomendación basada en modelos de ML](#elbow_curve)

[Calculamos la similitud entre usuarios y productos](#colaborative_filtering)

[Recomendación "user based"](#recomendacion_usuarios)

[Recomendación "product based"](#recomendacion_animes)

[Conclusión](#conclusión)

<a id = "import_modules"></a>
# Importación de las principales librerías
[Volver al índice](#toc)

En esta sección del kernel vamos a cargar las principales librerías que vamos a usar en nuestro notebook.

In [8]:
# silence warnings
import warnings
warnings.filterwarnings("ignore")

# operating system
import os

# time calculation to track some processes
import time

# numeric and matrix operations
import math
import numpy as np
import pandas as pd

# scientific computations library
import scipy as sp

# loading ploting libraries
import matplotlib.pyplot as plt
%matplotlib inline

# import the function to compute cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
anime_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Ejercicios/rating.csv")
rating=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Ejercicios/anime.csv")

<a id = "import_data"></a>
# Importación de los datos
[Volver al índice](#toc)

En la presente sección del kernel vamos a cargar los principales datasets que vamos a usar para construir nuestro recomendador.

In [11]:
#PATH_ANIME = os.path.join(PATH_FOLDER, 'cf_anime.parquet.gzip')

#anime_df = pd.read_parquet(PATH_ANIME)

In [12]:
#PATH_RATING = os.path.join(PATH_FOLDER, 'cf_rating.parquet.gzip')

 #rating_df = pd.read_parquet(PATH_RATING)

<a id = "eda"></a>
# Exploratory Data Analysis (EDA)
[Volver al índice](#toc)

En la sección del EDA haremos **una primera aproximación a nuestros datos** para ver su composición y que variables tenemos a nuestra disposición.

<a id = "df1"></a>
# EDA anime df
[Volver al índice](#toc)

EDA rápido sobre el **dataset de anime.**

In [13]:
def report_df(df, verbose = True):
    '''
    Hace un report simple sobre el DataFrame suministrado.
    '''
    print(df.info(verbose = verbose))
    total_nulos = df.isnull().sum().sum()
    print()
    print(f"Tenemos un total de {total_nulos} nulos")

In [14]:
report_df(anime_df)

anime_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
None

Tenemos un total de 0 nulos


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


Observamos que tenemos algunos nulos y tendremos que lidiar con ellos.

In [15]:
anime_df.isnull().sum()

,0
user_id,0
anime_id,0
rating,0


Por tipología de animes, vemos que el más popular es el de TV.

In [17]:
anime_df['type'].value_counts().plot(kind='bar', title = 'Anime by type')

KeyError: 'type'

In [ ]:
anime_df.shape

In [ ]:
anime_df = anime_df[anime_df['type'].isin(['TV', 'Movie'])]

In [ ]:
anime_df.shape

En nuestro report_df hemos visto que **episodes** parecía que era númerico, pero podría contener otro tipos de datos (por ser object), convertimos a número esta columna.

In [ ]:
anime_df['episodes'] = pd.to_numeric(anime_df['episodes'], errors = 'coerce')
anime_df['episodes'].fillna(1, inplace = True)

En la siguiente sección vamos a analizar la distribución de los animes en función del número de episodios que tenían.

In [ ]:
count_per_episodes = anime_df['episodes'].value_counts().to_frame().reset_index()

In [ ]:
count_per_episodes.columns = ['nr_episodes', 'nr_films']

In [ ]:
count_per_episodes

In [ ]:
count_per_episodes.sort_values('nr_episodes', ascending = True, inplace = True)

In [ ]:
count_per_episodes['pct_over_total'] = count_per_episodes['nr_films']/count_per_episodes['nr_films'].sum()

Casi la mitad de los animes es de un único episodio (41.14%).

In [ ]:
NR = 30

# instanciate the figure
fig = plt.figure(figsize = (15, 5))
ax = fig.add_subplot(111)

# separete the data
x = count_per_episodes["nr_episodes"].values[:NR]
y = count_per_episodes["nr_films"].values[:NR]
y_pct = count_per_episodes["pct_over_total"].values[:NR]

# plot the data
barplot = ax.bar(x, y)

# add text to each column
for rect, y_pct_ in zip(barplot, y_pct):
    y_pct_ = round(y_pct_*100, 2)
    height = rect.get_height()
    plt.text(
        rect.get_x() + rect.get_width()/2.0,
        height,
        f"{height}:{y_pct_}%",
        ha = 'center',
        va = "bottom",
        rotation = 60
    )

# change the xticks
ax.set_xticks(np.arange(0, NR + 1))

# add title
total_y_pct = round(sum(y_pct)*100, 2)
ax.set_title(f"Distribución de los primeros {NR} animes ({total_y_pct}% del total)");

Vamos a realizar un análisis similar al anterior, pero ahora veremos como se distribuyen los animes en función de la puntuación media.

Para tener sólo 10 grupos, primero vamos a redondear la puntuación media.

In [ ]:
anime_df['rating'].fillna(np.mean(anime_df['rating']), inplace = True)

In [ ]:
anime_df['ceil_rating'] = anime_df['rating'].apply(lambda rating: np.round(rating, 0))

In [ ]:
anime_df.tail(20)

In [ ]:
count_per_rating = anime_df['ceil_rating'].value_counts().to_frame().reset_index().sort_values('ceil_rating', ascending = True)

In [ ]:
count_per_rating

In [ ]:
# instanciate the figure
fig = plt.figure(figsize = (15, 5))
ax = fig.add_subplot(111)

# separete the data
x = count_per_rating["ceil_rating"].values
y = count_per_rating["count"].values
y_pct = y/sum(y)

# plot the data
barplot = ax.bar(x, y)

# add text to each column
for rect, y_pct_ in zip(barplot, y_pct):
    y_pct_ = round(y_pct_*100, 2)
    height = rect.get_height()
    plt.text(
        rect.get_x() + rect.get_width()/2.0,
        height,
        f"{height} - {y_pct_}%",
        ha = 'center',
        va = "bottom"
    )

ax.set_xticks(np.arange(0, 11))
ax.set_title("Distribución del número de animes por rating");

La puntuación más común es un 7 y se encuentra en 2.685 animes (43.77% del total).

<a id = "df2"></a>
# EDA ratings df
[Volver al índice](#toc)

EDA rápido sobre el DataFrame de **ratings de los animes.**

In [ ]:
report_df(rating_df)

rating_df.head()

In [ ]:
rating_gb = rating_df['rating'].value_counts().reset_index().sort_values('rating', ascending = True)

In [ ]:
rating_gb

In [ ]:
fig = plt.figure(figsize = (15, 5))
ax = fig.add_subplot()

x = rating_gb["rating"]
y = rating_gb["count"]
y_pct = y/sum(y)

barplot = ax.bar(x, y)

# add text to each column
for rect, y_pct_ in zip(barplot, y_pct):
    y_pct_ = round(y_pct_*100, 2)
    height = rect.get_height()
    plt.text(
        rect.get_x() + rect.get_width()/2.0,
        height,
        f"{height} - {y_pct_}%",
        ha = 'center',
        va = "bottom",
        rotation = 60
    )

ax.set_xticks(x);

Observamos que tenemos **casi un 19% de animes sin reviews.**

Vamos a ver los usuarios con más reviews.

In [ ]:
user_pivot = rating_df['user_id'].value_counts()

user_pivot.head()

Vamos a eliminar de nuestro DataFrame a todos aquellos usuarios cuyo único review es -1.

In [ ]:
s = rating_df.groupby('user_id')['rating'].apply(set)

In [ ]:
s

In [ ]:
user_id_no_reviews = s.to_frame()[s.to_frame()['rating'] == {-1}].index

In [ ]:
user_id_no_reviews

In [ ]:
rating_df = rating_df[-rating_df['user_id'].isin(user_id_no_reviews)]

In [ ]:
rating_df[rating_df['user_id'] == 73515]['rating'].value_counts()

<a id = "join"></a>
# Join final animes con ratings de usuarios
[Volver al índice](#toc)

Una vez que hemos analizado nuestros DataFrames, vamos a hacer un join **por anime_id.**

In [ ]:
rating_df.head(2)

In [ ]:
anime_df.head(2)

In [ ]:
rating_df.rename(columns = {'rating':'user_rating'}, inplace = True)
anime_df.rename(columns = {'rating':'average_rating'}, inplace = True)

In [ ]:
df_final = pd.merge(rating_df, anime_df, on = 'anime_id')

In [ ]:
df_final.head()

In [ ]:
df_final = df_final.pivot_table(
    index = 'user_id',
    columns = 'name',
    values = 'user_rating'
)

In [ ]:
df_final.head(20)

In [ ]:
df_final.fillna(-1, inplace = True)

In [ ]:
df_final.shape

<a id = "pca"></a>
# Reducción de la dimensionalidad con PCA
[Volver al índice](#toc)

El PCA es el método más utilizado de reducción de la dimensionalidad y con una largo desarrollo teórico detrás (fue [inventado en 1901](https://en.wikipedia.org/wiki/Principal_component_analysis) por Karl Pearson).


El PCA no requiere mantener las definiciones del espacio original de atributos. Intuitivamente, se basa en la siguiente idea: dada una colección de puntos en dos o más dimensiones, puede definirse una línea con un ajuste óptimo que minimice la suma de distancias cuadráticas de cada punto a la línea. Definida esta línea, se puede definir una nueva línea perpendicular a ésta y repetir este proceso hasta construir una base ortogonal que llamaremos Componentes Principales, en la cuál podemos expresar todas las variables de manera que un subconjunto reducido de componentes nos permite explicar la mayor parte de la varianza del problema original.

Vamos a ver un ejemplo muy sencillo en 2 dimensiones para coger la intuición detrás del PCA:

![PCA](https://drive.google.com/uc?export=view&id=14zcKhX9ICCpZ6BF0j3BFHAQsZMaXQjIa)

Cuando inicializamos el PCA de sklearn, le tenemos que especificar el número de componentes que queremos que tenga nuestro nuevo dataset.

In [ ]:
st = time.time()

pca = PCA(n_components = 30)
pca.fit(df_final)
pca_x = pca.transform(df_final)

pca_df = pd.DataFrame(
    data = pca_x,
    index = df_final.index,
    columns = ["PC_{}".format(i + 1) for i in range(30)]
)

et = time.time()
print("Total PCA took {} minutes".format(round((et - st)/60, 2)))

In [ ]:
pca_df.head()

Una vez fiteado el algoritmo de PCA, podemos mirar el átributo de **explained_variance_ratio_**.

Este átributo nos dice cuanta varianza explica cada uno de los componentes/columnas del nuevo dataset, del dataset original.

**Por ejemplo: el primer componente, contiene el 12% de la varianza original.**

In [ ]:
pca.explained_variance_ratio_

In [ ]:
sum(pca.explained_variance_ratio_)

**Con 30 componentes, explicamos el 36% de la varianza original. Puede parecer "poca" pero tenemos que tener en cuenta que pasamos de 4.957 columnas a 30.**

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Nr. PC')
plt.ylabel('Cumulative explained variance');

<a id = "elbow_curve"></a>
# Elbow curve, KMeans y recomendación basada en modelos de ML
[Volver al índice](#toc)

Dado que ahora podemos reducir nuestro dataset. Podemos utilizar el algoritmo KMeans y segmentar a nuestros clientes utilizando el pca_df.

In [ ]:
st = time.time()

sse = {}

for k in range(2, 8):
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(pca_df)
    sse[k] = kmeans.inertia_

et = time.time()
print("Total Elbow curve took {} minutes".format(round((et - st)/60, 2)))

In [ ]:
fig = plt.figure(figsize = (16, 8))
ax = fig.add_subplot()

x_values = list(sse.keys())
y_values = list(sse.values())

ax.plot(x_values, y_values, label = "Inertia/dispersión de los clústers")
ax.set_xticks(np.arange(1, 8))
fig.suptitle("Variación de la dispersión de los clústers en función de la k", fontsize = 16);

In [ ]:
kmeans = KMeans(n_clusters = 4, random_state = 175)

In [ ]:
kmeans.fit(pca_df)

In [ ]:
df_final['cluster'] = kmeans.labels_

In [ ]:
df_final.head()

Ahora tenemos, a nuestros clientes **segmentados** y podemos llegar a plantear un recommendador de animes, en base al clúster que pertenece cada persona.

In [ ]:
df_final.groupby('cluster').size()

In [ ]:
cluster_ = df_final[df_final['cluster'] == 1]

In [ ]:
cluster_.head()

In [ ]:
cluster_.drop('cluster', axis = 1, inplace = True)

In [ ]:
cluster_.replace([-1], np.nan, inplace = True)

**RECORDAD:** La función melt en pandas se utiliza para transformar un DataFrame de un formato amplio (wide) a un formato largo (long). Esto es particularmente útil cuando se desea reorganizar datos para que las variables sean mejor representadas para análisis o visualización.

En un DataFrame en formato amplio, cada variable tiene su propia columna. En el formato largo, hay dos columnas clave: una para las variables y otra para los valores.

In [ ]:
random_user = np.random.choice(cluster_.index)

In [ ]:
melted_df = cluster_.melt()

In [ ]:
melted_df.dropna(inplace = True)

In [ ]:
melted_df = melted_df[melted_df['name'] != 'cluster']

In [ ]:
average_score_cluster_ = melted_df.groupby('name').agg(
    score_medio = ('value', np.mean),
    nr_reviews = ('value', len)
)

In [ ]:
average_score_cluster_.head()

In [ ]:
average_score_cluster_.sort_values('nr_reviews', ascending = False, inplace = True)

In [ ]:
def build_recommendation_for_user_cluster_based(
    average_score_cluster_,
    cluster_,
    user_id,
    nr_recommendations = 10,
    verbose = True):
    '''
    Builds a personal recommendation for a user based on the best score in the cluster he belongs.
    '''
    print(f"Building recommendation for user_id {user_id}")
    print("----------------------------------------------------------------------------------------------\n")

    score_user = cluster_.loc[user_id]

    i = 0

    for anime_id in average_score_cluster_.index:

        score_medio_cluster_ = round(average_score_cluster_.loc[anime_id]["score_medio"], 2)
        nr_reviews_cluster_ = int(average_score_cluster_.loc[anime_id]["nr_reviews"])

        score_user_anime_ = score_user.loc[anime_id]

        if math.isnan(score_user_anime_):
            print(
                f'''
                {i + 1} Recommend this user {anime_id},
                anime score in the cluster is {score_medio_cluster_},
                nr reviews {nr_reviews_cluster_}\n
                '''
            )
            i += 1

            if i == nr_recommendations:
                break
        else:
            if verbose: print(f"User has seen this anime {anime_id}")

In [ ]:
build_recommendation_for_user_cluster_based(
    average_score_cluster_ = average_score_cluster_,
    cluster_ = cluster_,
    user_id = random_user
)

<a id = "colaborative_filtering"></a>
# Calculamos la similitud entre usuarios y productos
[Volver al índice](#toc)

**RECORDAD:** Introducir similitud del coseno con el otro notebook

En la presente sección vamos a calcular la similitud de coseno para los usuarios con más reviews.

Seleccionamos a los users con más reviews.

**Para poder hacer un recomendador correcto, necesitamos un dataset "denso". Es decir con usuarios que han puntuado mucho.**

Vamos a filtrar a aquellos que al menos tengan 350 reviews diferentes a -1.

In [ ]:
top_users_ratings = rating_df[rating_df['user_rating'] != -1]['user_id'].value_counts()

In [ ]:
(top_users_ratings >= 350).sum()

In [ ]:
top_users_for_cs = top_users_ratings[top_users_ratings >= 350].index

In [ ]:
top_users_for_cs

In [ ]:
df_final = df_final.reindex(index = top_users_for_cs)

In [ ]:
df_final

In [ ]:
df_final.drop('cluster', axis = 1, inplace = True)

In [ ]:
df_final.shape

In [ ]:
EXECUTE = False

In [ ]:
if EXECUTE:
    # normalizamos nuestro dataset
    st = time.time()
    df_final_norm = df_final.apply(lambda x: (x - np.mean(x))/(np.max(x) - np.min(x)), axis = 1)
    et = time.time()
    print("Normalization took {} minutes".format(round((et - st)/60, 2)))

    # esta vez vamos a imputar los nulos con 0, para que afecte menos en el cálculo de la similitud
    df_final_norm.fillna(0, inplace = True)
    sparse_ratings = sp.sparse.csr_matrix(df_final_norm.values)

    st = time.time()
    print("Working with user similarity")
    user_similarity = cosine_similarity(sparse_ratings)
    user_sim_df = pd.DataFrame(user_similarity, index = df_final_norm.index, columns = df_final_norm.index)
    user_sim_df.columns = map(str, user_sim_df.columns)
    user_sim_df.to_parquet("cf_user_similarity.parquet.gzip")

    print("Working with item similarity")
    item_similarity = cosine_similarity(sparse_ratings.T)
    item_sim_df = pd.DataFrame(item_similarity, index = df_final_norm.columns, columns = df_final_norm.columns)
    item_sim_df.columns = map(str, item_sim_df.columns)
    item_sim_df.to_parquet("cf_item_similarity.parquet.gzip")

    et = time.time()
    print("Total time to calculate similarity took {} minutes.".format(round((et - st)/60, 2)))

else:
    user_sim_df = pd.read_parquet(os.path.join("/content/drive/MyDrive/Nuclio_No_Supervisado/unsupervised_learning_extra_data/cf_user_similarity.parquet.gzip"))
    item_sim_df = pd.read_parquet(os.path.join("/content/drive/MyDrive/Nuclio_No_Supervisado/unsupervised_learning_extra_data/cf_item_similarity.parquet.gzip"))

In [ ]:
user_sim_df.head(20)

In [ ]:
item_sim_df.head()

<a id = "recomendacion_usuarios"></a>
# Recomendación "user based"
[Volver al índice](#toc)

Usando las similitudes antes calculadas, ahora podemos hacer recomendaciones a nuestros usuarios.

Para ello podemos seleccionar 1 usuario al azar, ver a los usuarios que mas se le parece y en función de los animes que le han gustado a este segundo user, hacer nuestra recomendación.

In [ ]:
def top_users(user, df):
    '''
    This function prints the top 10 similar users based on cosine similarity.
    '''

    df.columns = map(int, df.columns)

    if user not in df.columns:
        return('No data available on user {}'.format(user))

    print('Most Similar Users:\n')

    sim_users = df.sort_values(by = user, ascending=False).index[1:11]
    sim_values = df.sort_values(by = user, ascending=False).loc[:,user].tolist()[1:11]

    for user, sim in zip(sim_users, sim_values):
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim))

    return sim_users

In [ ]:
def compare_2_users(user1, user2, df, nr_animes):
    '''
    Returns a DataFrame with top 10 animes by 2 similar users (based on cosine similarity).
    '''

    top_10_user_1 = df[df.index == user1].melt().sort_values("value", ascending = False)[:nr_animes]
    top_10_user_1.columns = ["name_user_{}".format(user1), "rating_user_{}".format(user1)]
    top_10_user_1 = top_10_user_1.reset_index(drop = True)

    top_10_user_2 = df[df.index == user2].melt().sort_values("value", ascending = False)[:nr_animes]
    top_10_user_2.columns = ["name_user_{}".format(user2), "rating_user_{}".format(user2)]
    top_10_user_2 = top_10_user_2.reset_index(drop = True)

    combined_2_users = pd.merge(
        left = top_10_user_1,
        right = top_10_user_2,
        how = "outer",
        left_on = "name_user_{}".format(user1),
        right_on = "name_user_{}".format(user2)
    )

    return combined_2_users.dropna()

In [ ]:
user1 = 8250

similar_users = top_users(user1, user_sim_df)

In [ ]:
similar_users

In [ ]:
user2 = similar_users[0]

In [ ]:
combined_2_users = compare_2_users(user1, user2, df_final, 30)

In [ ]:
combined_2_users

<a id = "recomendacion_animes"></a>
# Recomendación "product based"
[Volver al índice](#toc)

También podemos llegar a hacer recomendaciones basadas en productos.

Por ejemplo, podemos llegar a buscar animes parecidos entre si en función de los reviews que han dejado los users.

In [ ]:
def top_animes(name, df):
    '''
    This functions prints top 10 similar animes, based on the reviews of the users.
    '''
    print('Similar shows to {} include:\n'.format(name))

    index = item_sim_df[name].sort_values(ascending = False).index[1:11]
    values = item_sim_df[name].sort_values(ascending = False).values[1:11]

    for i, (index_, values_) in enumerate(zip(index, values)):
        print('No. {}: {} ({})'.format(i + 1, index_, round(values_, 3)))

In [ ]:
top_animes('InuYasha', item_sim_df)

<a id = "conclusión"></a>
# Conclusión
[Volver al índice](#toc)

En el presente Notebook hemos explorado algunas de las técnicas más comunes que se utilizan en el unsupervised learning como: **PCA, KMeans.**

Posteriormente, hemos utilizado la métrica de **"cosine similarity"** para crear dos modelos de colaborative filtering: **user and product based.**

Hemos podido comprobar como las técnias de UL son muy útiles y se pueden utilizar en infinidad de campos desde: **visualización de datos, creación de nuevas variables (los componentes princiaples) y reducción de la dimensionalidad para agiliar el aprendizaje entre otros.**